## Portfolio Optimization Tools in Python

##### Author: Zachary Wright, CFA, FRM | Last Updated: 01/27/25

Welcome! This is a portfolio project dedicated toward showcasing skills acquired in Python in a quantitative finance scenario and may be updated as of time of viewing.

**Goal:** optimize a portfolio using various methods; maximize Sharpe ratio, minimize variance, and target return and risk.

Tools used:
- yfinance API to gather asset level data dynamically
- Scipy library to optimize asset weightings
- Python class constructor and object-oriented programming techniques for efficiency and modularity

First I will install and then import the required libraries:

In [1]:
!pip install pandas
!pip install numpy
!pip install yfinance
!pip install scipy

In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
from scipy.optimize import minimize

Next, since Python is an interpreted language, I will define a Portfolio class and its associated functions and methods first.
The class methods

In [3]:
#Create class portfolio - we can theoretically have multiple portfolios with different asset and return data series.
class Portfolio:
    def __init__(self, returns, weights):
        self.returns = returns
        self.weights = weights

    #Get return data from respective tickers - default interval is 1 month data.
    def get_returns_from_yfinance(self, tickers, start_date, end_date, interval="1mo"):
        all_data = {}
        for ticker in tickers:
            try:
                print(f"Fetching data for {ticker}...")
                data = yf.download(ticker, start=start_date, end=end_date, interval=interval)
                
                # Handle multi-index columns
                if isinstance(data.columns, pd.MultiIndex):
                    #Access the specific 'Close' column for this ticker
                    if ('Close', ticker) in data.columns:
                        prices = data[('Close', ticker)]
                    else:
                        print(f"No valid 'Close' column found for {ticker}. Skipping...")
                        continue
                else:
                    print(f"Unexpected column structure for {ticker}. Skipping...")
                    continue
                
                #Calculate percentage returns
                returns = prices.pct_change().dropna()
                all_data[ticker] = returns
            except Exception as e:
                print(f"Error fetching data for {ticker}: {e}")
        
        if all_data:
            return pd.DataFrame(all_data)
        else:
            raise ValueError("No valid data fetched. Please check the tickers or date range.")

    #Return the return of portfolio
    def get_return(self, weights=None):
        avg_asset_returns = self.returns.mean()
        if weights is None:
            weights = self.weights
        port_return = sum(w * avg_ret for w, avg_ret in zip(weights, avg_asset_returns))
        return port_return

    #Return the covariance of the portfolio
    def get_cov(self):
        return self.returns.cov()

    #Return the variance of the portfolio
    def get_var(self, weights=None):
        if weights is None:
            weights = self.weights
        cov_matrix = self.get_cov()
        weights = np.array(weights)
        port_variance = weights.T @ cov_matrix @ weights
        return port_variance

    #Optimization function: can maximize Sharpe Ratio, minimize variance, and target return and risk
    def optimize(self, objective="sharpe", target_return=None, target_risk=None):
        n_assets = len(self.weights)
        avg_returns = self.returns.mean()
        cov_matrix = self.get_cov()

        #Risk-free rate proxy from TBLL ETF yield through yfinance
        tbll = yf.Ticker("TBLL")
        rfr_proxy = tbll.info.get('dividendYield', 0)

        #Define constraints
        constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]
        bounds = [(0, 1) for _ in range(n_assets)]

        #Define the objective function
        if objective == "sharpe":
            def objective_function(weights):
                port_return = self.get_return(weights)
                port_variance = self.get_var(weights)
                sharpe_ratio = (port_return - rfr_proxy) / np.sqrt(port_variance)
                return -sharpe_ratio
        elif objective == "min_variance":
            def objective_function(weights):
                return self.get_var(weights)
        elif objective == "target_return":
            if target_return is None:
                raise ValueError("You must specify a target_return for this objective.")
            def objective_function(weights):
                return self.get_var(weights)
            constraints.append({'type': 'eq', 'fun': lambda w: self.get_return(w) - target_return})
        elif objective == "target_risk":
            if target_risk is None:
                raise ValueError("You must specify a target_risk for this objective.")
            def objective_function(weights):
                return abs(self.get_var(weights) - target_risk)
        else:
            raise ValueError("Invalid optimization objective.")

        #Optimize
        result = minimize(objective_function, self.weights, constraints=constraints, bounds=bounds)
        if result.success:
            self.weights = result.x
        return result

From here, we can import our portfolio's assets and their associated monthly returns. 

A portfolio object can now be created. You can see the optimized output below based on a maximized Sharpe ratio objective:

In [4]:
#Define tickers and date range
tickers = ["IVV", "IXUS", "IEMG"]
start_date = "2020-01-01"
end_date = "2023-12-31"

#Initialize dummy Portfolio object with placeholders
dummy_returns = pd.DataFrame()  # Placeholder returns (will be replaced)
dummy_weights = [1 / len(tickers)] * len(tickers)  # Equal weighting
portfolio = Portfolio(dummy_returns, dummy_weights)

#Fetch returns from Yahoo Finance
try:
    portfolio_returns = portfolio.get_returns_from_yfinance(tickers, start_date, end_date)
    print("Fetched Returns:")
    # print(portfolio_returns.head())
except ValueError as e:
    print(e)
    portfolio_returns = None

#Create a new Portfolio object with fetched returns
if portfolio_returns is not None:
    weights = [1 / len(tickers)] * len(tickers)  
    portfolio = Portfolio(portfolio_returns, weights)

    # Initial Portfolio Stats
    initial_return = portfolio.get_return() * 100
    initial_variance = portfolio.get_var()  
    print(f"Initial Portfolio Return: {initial_return:.2f}%")
    print(f"Initial Portfolio Variance: {initial_variance:.6f}")

    # Optimization Output
    optimized_result = portfolio.optimize(objective="sharpe")
    print("Optimized Weights: ")
    for ticker, weight in zip(tickers, optimized_result.x):
        print(f"  {ticker}: {weight * 100:.2f}%")
    
    optimized_return = portfolio.get_return(optimized_result.x) * 100
    optimized_variance = portfolio.get_var(optimized_result.x)  # Variance in raw terms
    print(f"Optimized Portfolio Return: {optimized_return:.2f}%")
    print(f"Optimized Portfolio Variance: {optimized_variance:.6f}")

Fetching data for IVV...


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Fetching data for IXUS...
Fetching data for IEMG...


[*********************100%***********************]  1 of 1 completed


Fetched Returns:
Initial Portfolio Return: 0.67%
Initial Portfolio Variance: 0.002996
Optimized Weights: 
  IVV: 100.00%
  IXUS: 0.00%
  IEMG: 0.00%
Optimized Portfolio Return: 1.13%
Optimized Portfolio Variance: 0.003447


Thank you for viewing this portfolio project!